In [3]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = {
    "EPOCH": 50, # number epochs
    "BATCH_SIZE": 50,
    "VALIDATION_BATCH_SIZE": 500,
    "LR": 0.01,
    "WEIGHT_DECAY": 4e-4,
    "LR_SCHEDULER": False,
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "AUGMENTATION_PROB": 0.0, # for random data transformation, see below (CIFAR-10 cells)
    "NORM_LAYER": "", # Normalization layer [None: "", Batch: "bn", Group: "gn"]
}

# Random transformations to provide data augmentation
random_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(1),
        transforms.ColorJitter(0.9, 0.9)
    ]
)

# Prepare the training set
train_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.RandomApply([random_transform], config["AUGMENTATION_PROB"]),
            transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]), # from the net, there are the values of cifer10
        ]
    ),
)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=config["BATCH_SIZE"], shuffle=True, num_workers=2
)

# Prepare the test set
test_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]),
        ]
    ),
)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=config["VALIDATION_BATCH_SIZE"], shuffle=False, num_workers=2
)

#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
